In [1]:
import ssl
ssl._create_default_https_context=ssl._create_unverified_context

In [2]:
import numpy as np
import pandas as pd

from functools import partial
from sklearn.datasets import fetch_openml
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_tweedie_deviance,
)


def load_mtpl2(n_samples=None):
    df_freq = fetch_openml(data_id=41214, as_frame=True).data
    df_freq["IDpol"] = df_freq["IDpol"].astype(int)
    df_freq.set_index("IDpol", inplace=True)

    df_sev = fetch_openml(data_id=41215, as_frame=True).data

    df_sev = df_sev.groupby("IDpol").sum()

    df = df_freq.join(df_sev, how="left")
    df["ClaimAmount"].fillna(0, inplace=True)

    for column_name in df.columns[df.dtypes.values == object]:
        df[column_name] = df[column_name].str.strip("'")
    return df.iloc[:n_samples]

def plot_obs_pred(
    df,
    feature,
    weight,
    observed,
    predicted,
    y_label=None,
    title=None,
    ax=None,
    fill_legend=False,
):
    df_ = df.loc[:, [feature, weight]].copy()
    df_["observed"] = df[observed] * df[weight]
    df_["predicted"] = predicted * df[weight]
    df_ = (
        df_.groupby([feature])[[weight, "observed", "predicted"]]
        .sum()
        .assign(observed=lambda x: x["observed"] / x[weight])
        .assign(predicted=lambda x: x["predicted"] / x[weight])
    )

    ax = df_.loc[:, ["observed", "predicted"]].plot(style=".", ax=ax)
    y_max = df_.loc[:, ["observed", "predicted"]].values.max() * 0.8
    p2 = ax.fill_between(
        df_.index,
        0,
        y_max * df_[weight] / df_[weight].values.max(),
        color="g",
        alpha=0.1,
    )
    if fill_legend:
        ax.legend([p2], ["{} distribution".format(feature)])
    ax.set(
        ylabel=y_label if y_label is not None else None,
        title=title if title is not None else "Train: Observed vs Predicted",
    )


def score_estimator(
    estimator,
    X_train,
    X_test,
    df_train,
    df_test,
    target,
    weights,
    tweedie_powers=None,
):
    """Evaluate an estimator on train and test sets with different metrics"""

    metrics = [
        ("D² explained", None),  # Use default scorer if it exists
        ("mean abs. error", mean_absolute_error),
        ("mean squared error", mean_squared_error),
    ]
    if tweedie_powers:
        metrics += [
            (
                "mean Tweedie dev p={:.4f}".format(power),
                partial(mean_tweedie_deviance, power=power),
            )
            for power in tweedie_powers
        ]

    res = []
    for subset_label, X, df in [
        ("train", X_train, df_train),
        ("test", X_test, df_test),
    ]:
        y, _weights = df[target], df[weights]
        for score_label, metric in metrics:
            if isinstance(estimator, tuple) and len(estimator) == 2:
                # Score the model consisting of the product of frequency and
                # severity models.
                est_freq, est_sev = estimator
                y_pred = est_freq.predict(X) * est_sev.predict(X)
            else:
                y_pred = estimator.predict(X)

            if metric is None:
                if not hasattr(estimator, "score"):
                    continue
                score = estimator.score(X, y, sample_weight=_weights)
            else:
                score = metric(y, y_pred, sample_weight=_weights)

            res.append({"subset": subset_label, "metric": score_label, "score": score})

    res = (
        pd.DataFrame(res)
        .set_index(["metric", "subset"])
        .score.unstack(-1)
        .round(4)
        .loc[:, ["train", "test"]]
    )
    return res


In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import (
    FunctionTransformer,
    KBinsDiscretizer,
    OneHotEncoder,
    StandardScaler,
)

df = load_mtpl2()

df.loc[(df["ClaimAmount"] == 0) & (df["ClaimNb"] >= 1), "ClaimNb"] = 0

df["ClaimNb"] = df["ClaimNb"].clip(upper=4)
df["Exposure"] = df["Exposure"].clip(upper=1)
df["ClaimAmount"] = df["ClaimAmount"].clip(upper=200000)

log_scale_transformer = make_pipeline(
    FunctionTransformer(func=np.log), StandardScaler()
)

column_trans = ColumnTransformer(
    [
        (
            "binned_numeric",
            KBinsDiscretizer(n_bins=10, subsample=int(2e5), random_state=0),
            ["VehAge", "DrivAge"],
        ),
        (
            "onehot_categorical",
            OneHotEncoder(),
            ["VehBrand", "VehPower", "VehGas", "Region", "Area"],
        ),
        ("passthrough_numeric", "passthrough", ["BonusMalus"]),
        ("log_scaled_numeric", log_scale_transformer, ["Density"]),
    ],
    remainder="drop",
)
X = column_trans.fit_transform(df)

df["PurePremium"] = df["ClaimAmount"] / df["Exposure"]

df["Frequency"] = df["ClaimNb"] / df["Exposure"]
df["AvgClaimAmount"] = df["ClaimAmount"] / np.fmax(df["ClaimNb"], 1)

with pd.option_context("display.max_columns", 15):
    print(df[df.ClaimAmount > 0].head())

/usr/local/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
/usr/local/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


       ClaimNb  Exposure Area  VehPower  VehAge  DrivAge  BonusMalus VehBrand  \
IDpol                                                                           
139.0      1.0      0.75    F       7.0     1.0     61.0        50.0      B12   
190.0      1.0      0.14    B      12.0     5.0     50.0        60.0      B12   
414.0      1.0      0.14    E       4.0     0.0     36.0        85.0      B12   
424.0      2.0      0.62    F      10.0     0.0     51.0       100.0      B12   
463.0      1.0      0.31    A       5.0     0.0     45.0        50.0      B12   

        VehGas  Density Region  ClaimAmount   PurePremium  Frequency  \
IDpol                                                                  
139.0  Regular  27000.0    R11       303.00    404.000000   1.333333   
190.0   Diesel     56.0    R25      1981.84  14156.000000   7.142857   
414.0  Regular   4792.0    R11      1456.55  10403.928571   7.142857   
424.0  Regular  27000.0    R11     10834.00  17474.193548   3.225806   


In [4]:
from sklearn.model_selection import train_test_split

df_train, df_test, X_train, X_test = train_test_split(df, X, random_state=0)

In [5]:
from sklearn.linear_model import TweedieRegressor

glm_pure_premium = TweedieRegressor(power=1.9, alpha=0.1, solver='newton-cholesky')
glm_pure_premium.fit(
    X_train, df_train["PurePremium"], sample_weight=df_train["Exposure"]
)

tweedie_powers = [1.5, 1.7, 1.8, 1.9, 1.99, 1.999, 1.9999]

scores_glm_pure_premium = score_estimator(
    glm_pure_premium,
    X_train,
    X_test,
    df_train,
    df_test,
    target="PurePremium",
    weights="Exposure",
    tweedie_powers=tweedie_powers,
)

scores = pd.concat(
    [scores_glm_pure_premium],
    axis=1,
    sort=True,
    keys=("TweedieRegressor"),
)
print("Evaluation of the Product Model and the Tweedie Regressor on target PurePremium")
with pd.option_context("display.expand_frame_repr", False):
    print(scores)

Evaluation of the Product Model and the Tweedie Regressor on target PurePremium
                                      T              
subset                            train          test
metric                                               
D² explained               1.690000e-02  1.420000e-02
mean Tweedie dev p=1.5000  7.640770e+01  7.640880e+01
mean Tweedie dev p=1.7000  3.682880e+01  3.692270e+01
mean Tweedie dev p=1.8000  3.037600e+01  3.045390e+01
mean Tweedie dev p=1.9000  3.382120e+01  3.387830e+01
mean Tweedie dev p=1.9900  2.015347e+02  2.015587e+02
mean Tweedie dev p=1.9990  1.914538e+03  1.914387e+03
mean Tweedie dev p=1.9999  1.904747e+04  1.904558e+04
mean abs. error            2.739865e+02  2.731249e+02
mean squared error         3.295505e+07  3.213056e+07


In [6]:
n_iter = glm_pure_premium.n_iter_


In [7]:
import secretflow as sf

print('The version of SecretFlow: {}'.format(sf.__version__))

sf.shutdown()

sf.init(['alice', 'bob'], address='local')

alice, bob = sf.PYU('alice'), sf.PYU('bob')
spu = sf.SPU(
    sf.utils.testing.cluster_def(
        ['alice', 'bob'],
        {"protocol": "REF2K", "field": "FM128", "fxp_fraction_bits": 40},
    ),
)

The version of SecretFlow: 1.6.1b0


/usr/local/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-08-06 09:15:22,016	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67104768 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=2.26gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-08-06 09:15:22,144	INFO worker.py:1724 -- Started a local Ray instance.


In [8]:
from secretflow.data import FedNdarray, PartitionWay

x, y = X_train, df_train["PurePremium"]
w = df_train["Exposure"]


def x_to_vdata(x):
    x = x.todense()
    v_data = FedNdarray(
        partitions={
            alice: alice(lambda: x[:, :15])(),
            bob: bob(lambda: x[:, 15:])(),
        },
        partition_way=PartitionWay.VERTICAL,
    )
    return v_data


v_data = x_to_vdata(x)

label_data = FedNdarray(
    partitions={alice: alice(lambda: y.values)()},
    partition_way=PartitionWay.VERTICAL,
)

sample_weight = FedNdarray(
    partitions={alice: alice(lambda: w.values)()},
    partition_way=PartitionWay.VERTICAL,
)

In [9]:
from secretflow.device.driver import reveal
from secretflow.ml.linear.ss_glm.core import get_dist

dist = 'Tweedie'
ss_glm_power = 1.9


class DirectRevealModel:
    def __init__(self, model) -> None:
        self.model = model

    def predict(self, X):
        vdata = x_to_vdata(X)
        y = self.model.predict(vdata)
        return reveal(y).reshape((-1,))

    def score(self, X, y, sample_weight=None):
        y = y.values
        y_pred = self.predict(X)

        constant = np.mean(y)
        if sample_weight is not None:
            constant *= sample_weight.shape[0] / np.sum(sample_weight)

        # Missing factor of 2 in deviance cancels out.
        deviance = get_dist(dist, 1, ss_glm_power).deviance(y_pred, y, None)
        deviance_null = get_dist(dist, 1, ss_glm_power).deviance(
            np.average(y, weights=sample_weight) + np.zeros(y.shape), y, None
        )
        return 1 - (deviance + constant) / (deviance_null + constant)

In [10]:
import time
from secretflow.ml.linear.ss_glm import SSGLM

model = SSGLM(spu)

ss_glm_power = 1.9
start = time.time()
model.fit_irls(
    v_data,
    label_data,
    None,
    sample_weight,
    2,
    'Log',
    'Tweedie',
    ss_glm_power,
    l2_lambda=0.1,
    infeed_batch_size_limit=10000000,
    fraction_of_validation_set=0.2,
    stopping_rounds=2,
    stopping_metric='deviance',
    stopping_tolerance=0.001,
)

wrapped_model = DirectRevealModel(model)

INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': 
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
INFO:root:irls calculating partials...
INFO:root:irls calculating partials...
INFO:root:irls calculating partials...
INFO:root:irls calculating partials...
INFO:root:irls updating weights...
INFO:root:epoch 1 train times: 12.741058826446533s
INFO:root:epoch 1 validation time cost: 1.1556816101074219
INFO:root:irls calculating partials...
INFO:root:irls calculating partials...
INFO:root:irls calculating partials...
INFO:root:irls calculating partials...
INFO:root:irls updating weights...
INFO:root:epoch 2 train times: 3.0125348567962646s
INFO:root:epoch 2 validation time cost: 0.19248199462890625


In [11]:
reveal(model.spu_w)


array([[ 3.41737620e-03],
       [ 1.89247951e-01],
       [ 5.89404225e-01],
       [-1.12057433e-01],
       [-1.91406030e-02],
       [ 3.76760662e-01],
       [-1.78643540e-01],
       [ 7.97630921e-02],
       [ 6.48671240e-02],
       [-6.64113700e-01],
       [ 3.58013153e-01],
       [-9.31122601e-01],
       [-4.33228731e-01],
       [-4.25371081e-02],
       [ 2.17337251e-01],
       [ 7.76762515e-02],
       [ 4.42767352e-01],
       [ 3.43533568e-02],
       [ 3.02862942e-01],
       [ 5.23277819e-01],
       [-2.29991794e-01],
       [ 7.61393070e-01],
       [ 6.59996927e-01],
       [-4.33898538e-01],
       [ 2.84868866e-01],
       [-5.70980132e-01],
       [ 1.28583670e-01],
       [ 1.76516414e-01],
       [ 1.00819051e-01],
       [-7.33127445e-02],
       [-6.76781118e-01],
       [-2.44344637e-01],
       [-5.11223078e-01],
       [-5.94665557e-02],
       [-1.31659880e-01],
       [-4.47464734e-01],
       [ 2.72815317e-01],
       [ 4.01874661e-01],
       [ 4.9

In [12]:
tweedie_powers = [1.5, 1.7, 1.8, 1.9, 1.99, 1.999, 1.9999]

scores_ss_glm_pure_premium = score_estimator(
    wrapped_model,
    X_train,
    X_test,
    df_train,
    df_test,
    target="PurePremium",
    weights="Exposure",
    tweedie_powers=tweedie_powers,
)


[2024-08-06 09:15:52.084] [info] [thread_pool.cc:30] Create a fixed thread pool with size 19


In [13]:
scores = pd.concat(
    [scores_glm_pure_premium, scores_ss_glm_pure_premium],
    axis=1,
    sort=True,
    keys=("TweedieRegressor", "SSGLMRegressor"),
)
print("Evaluation of the Tweedie Regressor and SS GLM on target PurePremium")
with pd.option_context("display.expand_frame_repr", False):
    print(scores)
    # print(scores.shape) # (10, 4)
    print(scores.columns)
    # print(scores['TweedieRegressor', 'train'])
    train_plain = scores['TweedieRegressor', 'train']
    train_cipher = scores['SSGLMRegressor', 'train']
    train_cmp = (train_cipher - train_plain) / train_plain
    print(f"明密文训练数据比较\n{train_cmp}")
    test_plain = scores['TweedieRegressor', 'test']
    test_cipher = scores['SSGLMRegressor', 'test']
    train_cmp = (test_cipher - test_plain) / test_plain
    print(f"明密文测试数据比较\n{train_cmp}")

Evaluation of the Tweedie Regressor and SS GLM on target PurePremium
                          TweedieRegressor                     SSGLMRegressor                       
subset                               train          test                train                   test
metric                                                                                              
D² explained                  1.690000e-02  1.420000e-02         -0.008159399          -0.0010857582
mean Tweedie dev p=1.5000     7.640770e+01  7.640880e+01           104.685472             107.990646
mean Tweedie dev p=1.7000     3.682880e+01  3.692270e+01            42.253085              42.932537
mean Tweedie dev p=1.8000     3.037600e+01  3.045390e+01            32.950962              33.348731
mean Tweedie dev p=1.9000     3.382120e+01  3.387830e+01            35.103795              35.341118
mean Tweedie dev p=1.9900     2.015347e+02  2.015587e+02           202.247704             202.379453
mean Tweedie dev p=1.9

In [14]:
res = []
for subset_label, x, df in [
    ("train", X_train, df_train),
    ("test", X_test, df_test),
]:
    exposure = df["Exposure"].values
    res.append(
        {
            "subset": subset_label,
            "observed": df["ClaimAmount"].values.sum(),
            "predicted, tweedie, power=%.2f" % glm_pure_premium.power: np.sum(
                exposure * glm_pure_premium.predict(x)
            ),
            "predicted, ss glm, power=%.2f" % ss_glm_power: np.sum(
                exposure * wrapped_model.predict(x)
            ),
        }
    )
print(pd.DataFrame(res).set_index("subset").T)


subset                                 train          test
observed                        3.917618e+07  1.299546e+07
predicted, tweedie, power=1.90  3.951751e+07  1.325198e+07
predicted, ss glm, power=1.90   1.250159e+09  2.951692e+09
